In [19]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [20]:
df = pd.read_csv('./data/train.csv')

In [21]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [22]:
new_df =  df.sample(30000)

In [23]:
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [24]:
ques_df = new_df[['question1','question2']]
ques_df.head()

,question1,question2
44658,"Why do we have 28 or 29 days in February, wher...",Fee stracture about Areena & framebox?
265409,What is your favorite game winning play?,What is your favorite game?
382061,Which car services are available in Sacramento...,Which car services are available in Houston? H...
32078,"What are of the pros and cons of both, the hus...",My wife is insisting upon going to a male gyne...
215257,How should I prepare for RBI grade B exam 2017?,Should I prepare for GMAT or RBI Grade B?


In [25]:
from sklearn.feature_extraction.text import CountVectorizer
  
#merge texts
questions = list(ques_df['question1'])+ list(ques_df['question2'])
#BOW
cv = CountVectorizer(max_features=3000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [26]:
temp_df1 = pd.DataFrame(q1_arr, index= ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index = ques_df.index)
temp_df = pd.concat([temp_df1,temp_df2], axis=1)
temp_df.shape

(30000, 6000)

In [27]:
# 30000 rows and (6000-> 3000 features of ques1 and 3000, features of ques2)
temp_df

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
44658,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
265409,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
382061,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32078,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
215257,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184513,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
270295,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
112444,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
342502,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
temp_df['is_duplicate'] = new_df['is_duplicate']

In [29]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
44658,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
265409,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
382061,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32078,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
215257,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
# NOW WE HAVE OUR SPARE MATRIX DATASET CONTAING THE BOW OF BOTH THE QUESTION AND THE TARGET VALUE

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(temp_df.iloc[:,0:-1].values,temp_df.iloc[:,-1].values,test_size=0.2,random_state=1)


In [32]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.736

In [34]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
accuracy_score(y_test,y_pred)

0.7198333333333333